In [61]:
KAGGLE = False

import os
import pandas as pd
import numpy as np
from functools import reduce
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

In [62]:
# !pip install catboost

In [63]:
import warnings
warnings.filterwarnings("ignore")

In [64]:

DATA_PATH = ''


SEED = 2025
cat_feats = []

In [65]:
train_data = pd.read_parquet(os.path.join(DATA_PATH, 'track3_train.parquet'))
test_data = pd.read_parquet(os.path.join(DATA_PATH, 'track3_a.parquet'))

train_data['type'] = 'train'
test_data['type'] = 'test'
data = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)
data = pd.concat([data, data['ID'].str.split('_', expand=True).rename(columns={0:'station',1:'sample',2:'time'})], axis=1)#细化向关联项

print(f'train_data: {train_data.shape}')
print(f'test_data: {test_data.shape}')
print(f'all: {data.shape}')


train_data: (791616, 77)
test_data: (216384, 74)
all: (1008000, 80)


In [66]:
TARGET_FEATS = ['wdir_2min', 'spd_2min', 'spd_inst_max']

In [67]:
# 删除有问题的目标
for item in TARGET_FEATS:
    idx = data[item] >= 199999.0
    data.loc[idx, item] = np.nan

In [68]:
data['station'] = data['station'].apply(lambda x: int(x.split('D')[-1]))
cat_feats += ['station']

data['sample'] = data['sample'].apply(lambda x: int(x.split('Sample')[-1]))
data['time'] = data['time'].astype(int)

In [69]:
feats = [item for item in data.columns if item not in TARGET_FEATS+['ID', 'sample', 'type']]#剔除不相关项
print(f'feats: {len(feats)}, {feats}')
print(cat_feats)

feats: 74, ['100u', '100v', '10u', '10v', '2d', '2t', 'cape', 'capes', 'cp', 'deg0l', 'lcc', 'msl', 'skt', 'sp', 'sst', 'tcc', 'd_L1000', 'q_L1000', 'r_L1000', 't_L1000', 'u_L1000', 'v_L1000', 'w_L1000', 'd_L950', 'q_L950', 'r_L950', 't_L950', 'u_L950', 'v_L950', 'w_L950', 'd_L925', 'q_L925', 'r_L925', 't_L925', 'u_L925', 'v_L925', 'w_L925', 'd_L900', 'q_L900', 'r_L900', 't_L900', 'u_L900', 'v_L900', 'w_L900', 'd_L850', 'q_L850', 'r_L850', 't_L850', 'u_L850', 'v_L850', 'w_L850', 'd_L700', 'q_L700', 'r_L700', 't_L700', 'u_L700', 'v_L700', 'w_L700', 'd_L500', 'q_L500', 'r_L500', 't_L500', 'u_L500', 'v_L500', 'w_L500', 'd_L200', 'q_L200', 'r_L200', 't_L200', 'u_L200', 'v_L200', 'w_L200', 'station', 'time']
['station']


In [70]:
data

,ID,wdir_2min,spd_2min,spd_inst_max,100u,100v,10u,10v,2d,2t,...,q_L200,r_L200,t_L200,u_L200,v_L200,w_L200,type,station,sample,time
0,D0011_Sample00000_01,88.0,4.5,5.2,-3.178867,1.520631,-3.035558,0.921220,-4.296475,2.166330,...,0.005066,7.282292,-54.574743,60.027509,4.458829,0.052047,train,11,0,1
1,D0011_Sample00000_02,101.0,4.5,6.0,-3.044837,1.737420,-2.924957,1.208521,-3.802847,2.137752,...,0.005006,6.950891,-54.490239,58.961815,3.799950,0.053916,train,11,0,2
2,D0011_Sample00000_03,111.0,3.9,5.1,-2.910807,1.954209,-2.814357,1.495822,-3.309219,2.109175,...,0.004946,6.619490,-54.405734,57.896121,3.141070,0.055785,train,11,0,3
3,D0011_Sample00000_04,160.0,1.7,4.3,-1.870547,1.442103,-1.843161,1.229792,-3.500122,1.850405,...,0.004919,6.931894,-54.718634,57.157805,2.129127,0.064269,train,11,0,4
4,D0011_Sample00000_05,179.0,1.3,2.9,-0.830286,0.929998,-0.871964,0.963762,-3.691025,1.591636,...,0.004893,7.244298,-55.031533,56.419490,1.117184,0.072752,train,11,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007995,D0009_Sample02128_44,NaN,NaN,NaN,3.843052,8.813500,3.456899,8.086399,-6.134135,4.715720,...,0.005002,7.706795,-54.882592,58.070532,-1.666188,0.017625,test,9,2128,44
1007996,D0009_Sample02128_45,NaN,NaN,NaN,3.674979,9.288052,3.270604,8.504519,-6.362700,5.081217,...,0.005217,8.115371,-54.979291,58.097675,-2.525959,-0.009414,test,9,2128,45
1007997,D0009_Sample02128_46,NaN,NaN,NaN,4.163034,9.299423,3.684067,8.492985,-5.268479,5.118454,...,0.005117,7.934781,-54.952597,58.558392,-3.415087,0.020155,test,9,2128,46
1007998,D0009_Sample02128_47,NaN,NaN,NaN,4.651088,9.310794,4.097529,8.481452,-4.174258,5.155692,...,0.005016,7.754191,-54.925903,59.019110,-4.304215,0.049725,test,9,2128,47


In [71]:
train_data = data.query('type=="train"').reset_index(drop=True)
test_data = data.query('type=="test"').reset_index(drop=True)

In [72]:
cv_params = {'iterations': [500,600,700,800]}
other_params = {
    'iterations': 90_000,
    'learning_rate':0.03,
    'l2_leaf_reg':3,
    'bagging_temperature':0.8,
    'random_strength':0.8,
    'rsm':0.8,
    'one_hot_max_size':2,
    'leaf_estimation_method':'Gradient',
    'fold_len_multiplier':3,
    'border_count':100,
}


In [73]:
# 逐小时预报采用四舍五入保留1位小数位
# 24小时内最大风由保留1位小数位的逐小时平均风速求最大值
# 24小时内极大风由保留1位小数位的逐小时极大风速求最大值

In [74]:
# task_name = "xgb"
# task_params = {"xgb": xgb_params}[task_name] # 用于控制模型训练的超参数字典，如学习率、树的深度、子采样比例等


In [75]:
from sklearn.preprocessing import LabelEncoder

train_oof = {}
test_pred = {}
feats_importance = {}

for item_target in TARGET_FEATS:
    train_y = train_data[item_target]
    idx = ~train_y.isna()

    trian_id = train_data.loc[idx, 'ID'].reset_index(drop=True)  #原始训练集中ID列+为缺失值的结果列
    train_x = train_data.loc[idx, feats].reset_index(drop=True)  #训练集（已细化并剔除不相关）+为缺失值的结果列
    testA_x = test_data[feats].reset_index(drop=True) #测试集（已细化并剔除不相关）
    train_y = train_y.loc[idx].reset_index(drop=True) #缺失值的结果列
    group_x = train_data.loc[idx, 'sample'].reset_index(drop=True) #原始训练集中sample列+为缺失值的结果列
    print(train_x.shape, testA_x.shape)

    item_oof = np.zeros(train_x.shape[0])
    item_pred = np.zeros(testA_x.shape[0])

    fold_num = 5
    item_importance = 0
    from sklearn.model_selection import GroupKFold #用于分组数据的交叉验证迭代器 测试模型的泛化能力 检测过拟合情况
    kf = GroupKFold(n_splits=fold_num, ) #n_splits一般3-10之间  数据集越大n_splits调小  越大越精准但泛化能力下降过拟合增大   需要找平衡点

    # testA_x=xgb.DMatrix(testA_x)

    for fold, (train_idx, val_idx) in enumerate(kf.split(train_x, groups=group_x)): #基于GroupKFold交叉验证分组，根据sample标识（文件夹）分组  保证每个分组样本均匀分配到不同训练集和验证集中
        print('-----------', fold)

        le = LabelEncoder()
        for col in cat_feats:
            train_x.loc[train_idx][col] = le.fit_transform(train_x.loc[train_idx][col])
            train_x.loc[val_idx][col] = le.transform(train_x.loc[val_idx][col])

        # train = xgb.DMatrix(train_x.loc[train_idx], label=train_y.loc[train_idx])
        # val = xgb.DMatrix(train_x.loc[val_idx], label=train_y.loc[val_idx])
        # watchlist = [(train, 'train'), (val, 'val')]
        train_x1 = train_x.loc[train_idx]
        train_y1 = train_y.loc[train_idx]
        val_x1   = train_x.loc[val_idx]
        val_y1   = train_y.loc[val_idx]


        model_cb = CatBoostRegressor(**other_params,
                                     task_type='GPU',
                                     loss_function='Quantile',
                                     eval_metric='Quantile',
                                     random_seed=SEED,
                                     od_type='Iter',
                                     )
        model_cb.fit(train_x1, train_y1,
                     eval_set=(val_x1, val_y1),
                     plot=True)
        """
        params: 用于控制模型训练的超参数字典，如学习率、树的深度、子采样比例等。
        train_set: 训练数据集，可以是 lgb.Dataset 对象或数据矩阵。
        num_boost_round: 迭代次数，即弱分类器的数量。
        valid_sets: 验证数据集，可以是一个 lgb.Dataset 对象或一个列表，其中每个元素都是 lgb.Dataset 对象。
        early_stopping_rounds: 早停次数，如果在连续的 early_stopping_rounds 次迭代中验证集的损失没有得到改善，则停止迭代。
            lgb.early_stopping()比这个好用，可以把num_boost_round放的很大，他可以智能判断早停
        callbacks: 回调函数列表，用于在训练过程中执行特定操作，如打印日志、保存模型等。
        """
        item_oof_x=train_x.loc[val_idx]
        item_oof[val_idx] += (model_cb.predict(item_oof_x)) #预测验证集结果存入item_oof

        item_pred += (model_cb.predict(testA_x))/fold_num  #预测测试集/折数=平局值 item_pred
        # item_importance += model.feature_importance(importance_type='gain') / fold_num # 特征性/折数=平均值 importance_type=split/gain 分别表示特征被用于分裂的次数和特征对模型的贡献度

    # importance = pd.DataFrame()
    # importance['name'] = feats
    # importance['importance'] = item_importance

    train_oof[item_target] = pd.DataFrame({"ID": trian_id, f"{item_target}_true": train_y, f"{item_target}_pred": item_oof}) #重构预测的验证集结果列
    test_pred[item_target] = pd.DataFrame({"ID": test_data['ID'], f"{item_target}": item_pred})
    # feats_importance[item_target] = importance #重构预测的测试集结果列

(770742, 74) (216384, 74)
----------- 0


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 45.2641378	test: 44.0902595	best: 44.0902595 (0)	total: 47ms	remaining: 4.65s
1:	learn: 45.2607237	test: 44.0868653	best: 44.0868653 (1)	total: 82.4ms	remaining: 4.04s
2:	learn: 45.2572746	test: 44.0834354	best: 44.0834354 (2)	total: 116ms	remaining: 3.76s
3:	learn: 45.2538125	test: 44.0799854	best: 44.0799854 (3)	total: 151ms	remaining: 3.62s
4:	learn: 45.2503761	test: 44.0765601	best: 44.0765601 (4)	total: 187ms	remaining: 3.54s
5:	learn: 45.2469613	test: 44.0731600	best: 44.0731600 (5)	total: 223ms	remaining: 3.49s
6:	learn: 45.2434994	test: 44.0697096	best: 44.0697096 (6)	total: 258ms	remaining: 3.43s
7:	learn: 45.2400842	test: 44.0663137	best: 44.0663137 (7)	total: 308ms	remaining: 3.55s
8:	learn: 45.2366312	test: 44.0628637	best: 44.0628637 (8)	total: 344ms	remaining: 3.48s
9:	learn: 45.2331977	test: 44.0594336	best: 44.0594336 (9)	total: 380ms	remaining: 3.42s
10:	learn: 45.2297786	test: 44.0560287	best: 44.0560287 (10)	total: 416ms	remaining: 3.37s
11:	learn: 45.22637

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 45.1018292	test: 44.7388583	best: 44.7388583 (0)	total: 43.5ms	remaining: 4.31s
1:	learn: 45.0984111	test: 44.7354514	best: 44.7354514 (1)	total: 79.7ms	remaining: 3.91s
2:	learn: 45.0949584	test: 44.7320137	best: 44.7320137 (2)	total: 115ms	remaining: 3.71s
3:	learn: 45.0914951	test: 44.7285428	best: 44.7285428 (3)	total: 153ms	remaining: 3.66s
4:	learn: 45.0880057	test: 44.7250861	best: 44.7250861 (4)	total: 187ms	remaining: 3.56s
5:	learn: 45.0845890	test: 44.7216856	best: 44.7216856 (5)	total: 224ms	remaining: 3.51s
6:	learn: 45.0811489	test: 44.7182674	best: 44.7182674 (6)	total: 261ms	remaining: 3.47s
7:	learn: 45.0777484	test: 44.7148860	best: 44.7148860 (7)	total: 298ms	remaining: 3.42s
8:	learn: 45.0742576	test: 44.7114144	best: 44.7114144 (8)	total: 342ms	remaining: 3.46s
9:	learn: 45.0708184	test: 44.7079920	best: 44.7079920 (9)	total: 383ms	remaining: 3.45s
10:	learn: 45.0673957	test: 44.7045898	best: 44.7045898 (10)	total: 420ms	remaining: 3.4s
11:	learn: 45.0639

...

0:	learn: 1.5348884	test: 1.6105509	best: 1.6105509 (0)	total: 38.3ms	remaining: 3.79s
1:	learn: 1.5322286	test: 1.6078605	best: 1.6078605 (1)	total: 66.6ms	remaining: 3.26s
2:	learn: 1.5295847	test: 1.6051713	best: 1.6051713 (2)	total: 94.2ms	remaining: 3.05s
3:	learn: 1.5269828	test: 1.6025380	best: 1.6025380 (3)	total: 125ms	remaining: 2.99s
4:	learn: 1.5243299	test: 1.5998409	best: 1.5998409 (4)	total: 152ms	remaining: 2.89s
5:	learn: 1.5216727	test: 1.5971454	best: 1.5971454 (5)	total: 180ms	remaining: 2.82s
6:	learn: 1.5190169	test: 1.5944609	best: 1.5944609 (6)	total: 209ms	remaining: 2.77s
7:	learn: 1.5163636	test: 1.5917777	best: 1.5917777 (7)	total: 237ms	remaining: 2.73s
8:	learn: 1.5137435	test: 1.5891179	best: 1.5891179 (8)	total: 265ms	remaining: 2.68s
9:	learn: 1.5111319	test: 1.5864763	best: 1.5864763 (9)	total: 295ms	remaining: 2.65s
10:	learn: 1.5085269	test: 1.5838456	best: 1.5838456 (10)	total: 323ms	remaining: 2.61s
11:	learn: 1.5059113	test: 1.5811968	best: 1.5811

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.5392062	test: 1.5933294	best: 1.5933294 (0)	total: 37.5ms	remaining: 3.72s
1:	learn: 1.5365607	test: 1.5906325	best: 1.5906325 (1)	total: 65.8ms	remaining: 3.22s
2:	learn: 1.5339088	test: 1.5879375	best: 1.5879375 (2)	total: 94.3ms	remaining: 3.05s
3:	learn: 1.5312638	test: 1.5852521	best: 1.5852521 (3)	total: 124ms	remaining: 2.98s
4:	learn: 1.5286149	test: 1.5825602	best: 1.5825602 (4)	total: 152ms	remaining: 2.89s
5:	learn: 1.5259537	test: 1.5798544	best: 1.5798544 (5)	total: 181ms	remaining: 2.83s
6:	learn: 1.5233077	test: 1.5771706	best: 1.5771706 (6)	total: 209ms	remaining: 2.77s
7:	learn: 1.5206524	test: 1.5744887	best: 1.5744887 (7)	total: 237ms	remaining: 2.73s
8:	learn: 1.5180226	test: 1.5718211	best: 1.5718211 (8)	total: 266ms	remaining: 2.68s
9:	learn: 1.5153792	test: 1.5691374	best: 1.5691374 (9)	total: 296ms	remaining: 2.67s
10:	learn: 1.5127479	test: 1.5664655	best: 1.5664655 (10)	total: 326ms	remaining: 2.63s
11:	learn: 1.5101272	test: 1.5638026	best: 1.5638

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.5565165	test: 1.5240013	best: 1.5240013 (0)	total: 39.5ms	remaining: 3.91s
1:	learn: 1.5538582	test: 1.5213819	best: 1.5213819 (1)	total: 67.4ms	remaining: 3.3s
2:	learn: 1.5511953	test: 1.5187731	best: 1.5187731 (2)	total: 95.9ms	remaining: 3.1s
3:	learn: 1.5485278	test: 1.5161561	best: 1.5161561 (3)	total: 126ms	remaining: 3.02s
4:	learn: 1.5458540	test: 1.5135355	best: 1.5135355 (4)	total: 155ms	remaining: 2.94s
5:	learn: 1.5431817	test: 1.5109061	best: 1.5109061 (5)	total: 183ms	remaining: 2.87s
6:	learn: 1.5405283	test: 1.5082962	best: 1.5082962 (6)	total: 212ms	remaining: 2.81s
7:	learn: 1.5378536	test: 1.5056595	best: 1.5056595 (7)	total: 240ms	remaining: 2.76s
8:	learn: 1.5352191	test: 1.5030683	best: 1.5030683 (8)	total: 269ms	remaining: 2.72s
9:	learn: 1.5325795	test: 1.5004804	best: 1.5004804 (9)	total: 298ms	remaining: 2.68s
10:	learn: 1.5299534	test: 1.4978921	best: 1.4978921 (10)	total: 326ms	remaining: 2.64s
11:	learn: 1.5273045	test: 1.4952915	best: 1.495291

In [76]:
train_oof = reduce(lambda x,y: pd.merge(x,y,on='ID', how='outer'), train_oof.values())
test_pred = reduce(lambda x,y: pd.merge(x,y,on='ID', how='outer'), test_pred.values())
# train_oof.shape
# test_pred.shape

score_str = 'Potato_c'

In [77]:
if not os.path.exists("result"):
    os.makedirs("result")
train_oof.to_csv(os.path.join("result", f'lgb_oof_{score_str}.csv'), index=False)
test_pred.to_csv(os.path.join("result", f'{score_str}.csv'), index=False)